In [33]:
from motion_tracking import MotionTracker
import utils
try:
    import cv2
except Exception as e:
    print("Warning: OpenCV not installed. To use motion detection, make sure you've properly configured OpenCV.")
import time
import torch
import torchvision.transforms as TT
import torch.nn.functional as F
import matplotlib.pyplot as plt


def get_gray_original(frame):
    H, W = frame.shape[:2]
    # ratio of new width to old width
    r = 500/float(W)
    print(frame.shape)
    frame = cv2.resize(frame, dsize=(500, int(H*r)), interpolation=cv2.INTER_LINEAR)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    sigma = [0.3*((21-1)*0.5 - 1) + 0.8 for i in range(2)]
    print(sigma)
    print(gray)
    #frame = utils.ndarray_to_tensor(frame)
    print(gray.shape)
    #plt.imshow(frame)
    #plt.show()

def get_gray_new(frame):
    #https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # can use size=500 keep it the same as before - was around a 78% dilation, but I changed it to 75%
    resize_func = TT.Resize(size=(360, 480), interpolation=TT.InterpolationMode.BILINEAR, antialias=True)
    gray_func = TT.Grayscale(num_output_channels=1)
    blur_func = TT.GaussianBlur((21,21), sigma=3.5) # what the original sigma came out to be with k=21 in cv2.GaussianBlur
    #print(frame)
    frame = utils.ndarray_to_tensor(frame)
    frame = resize_func(frame)
    gray = gray_func(frame)
    gray = blur_func(gray)
    print(gray)
    print(gray.shape)


def init_first_frame_old(gray):
    print("Waiting for video to adjust...")
    if tempFrame is None:
        tempFrame = gray
        return None
    else:
        delta = cv2.absdiff(tempFrame, gray)
        tempFrame = gray
        tst = cv2.threshold(delta, 5, 255, cv2.THRESH_BINARY)[1]
        tst = cv2.dilate(tst, None, iterations=2)
        if count > 30:
            print("Done.\n Waiting for motion.")
            if not cv2.countNonZero(tst) > 0:
                return gray
        else:
            count += 1
            return None

def init_first_frame_new(gray):
    pass


# gonna rewrite this outside of a Jupyter notebook in the morning

# default camera port seems to be 0
camera = cv2.VideoCapture(0)
time.sleep(0.25)
# initialize the first frame in the video stream
firstFrame = None
tempFrame = None
count = 0
# loop over the frames of the video
while True:
    (grabbed, frame) = camera.read()
    if not grabbed:
        break
    gray = get_gray_new(frame)
    if firstFrame is None:
        print("Waiting for video to adjust...")
        if tempFrame is None:
            tempFrame = gray
            continue
        else:
            delta = cv2.absdiff(tempFrame, gray)
            tempFrame = gray
            tst = cv2.threshold(delta, 5, 255, cv2.THRESH_BINARY)[1]
            tst = cv2.dilate(tst, None, iterations=2)
            if count > 30:
                print("Done.\n Waiting for motion.")
                if not cv2.countNonZero(tst) > 0:
                    firstFrame = gray
                else:
                    continue
            else:
                count += 1
                continue

test_new(frame)
#test_original(frame)
#frame = resize_func(frame)
#print(frame.shape)
#print(frame)
#plt.imshow(frame)
#plt.show()


tensor([[[  7,   7,   7,  ...,  31,  32,  32],
         [  7,   7,   7,  ...,  31,  32,  32],
         [  7,   7,   7,  ...,  31,  31,  31],
         ...,
         [129, 129, 127,  ...,  72,  72,  72],
         [128, 128, 126,  ...,  72,  72,  72],
         [128, 127, 126,  ...,  71,  71,  71]]], dtype=torch.uint8)
torch.Size([1, 360, 480])
